# Import modules

In [ ]:
import requests
import pandas as pd
import pickle
from bs4 import BeautifulSoup

# Open your text file and combine the paragraphs

In [ ]:
file = open("your_text_file_name_here.txt","r", encoding='utf8') 
text = '\n'.join(file)
print(text)

# Cleaning the text

In [ ]:
# Make text lowercase, remove the name, remove text in square brackets, remove punctuation and remove words containing numbers.
import re
import string

def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Check the result

In [ ]:
round1 = clean_text_round1(text)
print(round1)

# More cleaning if necessary, e.g.,

In [ ]:
# Get rid of some additional punctuation and non-sensical text that was missed the first time around.
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

# Check again

In [ ]:
round2 = clean_text_round2(round1)
print(round2)

# Save the contents as a text file with pickled data

In [ ]:
f = open('whatever_the_name_you_like.txt','wb')
pickle.dump(round2, f)
f.close()

# Load the pickled files

In [ ]:
data = open("whatever_the_name_you_like.txt", "r")

In [ ]:
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

# Make a dataframe with what we've got.

# Convert the dataframe into a document-term matrix

In [ ]:
# Also, exclude common words English stop words from the index. 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_df.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_df.index
data_dtm

# Pickle the data

In [ ]:
import pickle
data_dtm.to_pickle("dtm.pkl")
data_df.to_pickle('df.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

# Transpose the data for easy manipulation

In [ ]:
data = pd.read_pickle('dtm.pkl')
data = data.transpose()
data.head()

# Find the top 30 words used

# Create word clouds

In [ ]:
# To download a necessary module, open command prompt and run the comand "conda install -c conda-forge wordcloud"
from wordcloud import WordCloud
wc = WordCloud(background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [160, 60]

full_names = ['Joe Biden', 'Donald Trump']

# Create subplots for each
for index, name in enumerate(data.columns):
    wc.generate(data_df.transcript[name])
    
    plt.subplot(3, 4, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(full_names[index])

plt.savefig("figure.png")
plt.show()